# Выявление негативных комментарий с BERT

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.


**Цель проекта:**

Обучить модель классифицировать комментарии на позитивные и негативные на основе набора данных с разметкой о токсичности правок. Метрика качества *F1* должна быть не меньше 0.75. 

**Задачи:**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></ul></div>

## Подготовка

In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier

In [4]:
torch.cuda.is_available()

True

In [5]:
df = pd.read_csv('/kaggle/input/toxic-coments/toxic_comments.csv')
display(df.head(10))
df.info(memory_usage='deep')

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 80.1 MB


Загрузим предварительно обученную модель и токенизатор BERT. В данном случае используется модель, которая была обучена на выявление токсичных комментариев (toxic-bert).

In [6]:
model_class, tokenizer_class, pretrained_weights = (transformers.BertModel,
                                                    transformers.BertTokenizer,
                                                    'unitary/toxic-bert')


tokenizer = tokenizer_class.from_pretrained(pretrained_weights)

model = model_class.from_pretrained(pretrained_weights)

tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Произведем токенизацию текстов и создадим маску

In [7]:
tokenized = df['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Извелечем эмбеддинги используя GPU

In [8]:
%%time
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda()
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)

        embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())
        del batch
        del attention_mask_batch
        del batch_embeddings

  0%|          | 0/1595 [00:00<?, ?it/s]

CPU times: user 1h 11min 38s, sys: 2.18 s, total: 1h 11min 40s
Wall time: 1h 11min 38s


In [9]:
features = np.concatenate(embeddings)

Выделим обучающую и тестовую выборки

In [10]:
target = df['toxic']
target = target.iloc[:len(features)]

In [11]:
features_train, features_test, target_train, target_test = (
    train_test_split(features, target, test_size=0.2, random_state=42))

## Обучение

Для обучения используем модели LogisticRegression, DecisionTreeClassifier, CatBoost.

**Модель LogisticRegression**

In [12]:
%%time
model_regression = LogisticRegression(C = 12, random_state=42)
model_regression.fit(features_train, target_train)
predict = model_regression.predict(features_test)
f1_lr = f1_score(target_test, predict)

print(f"Показатель f1 на тестовой выборке: {f1_lr:.2f}")

Показатель f1 на тестовой выборке: 0.95
CPU times: user 30.6 s, sys: 2.7 s, total: 33.3 s
Wall time: 8.64 s


**Модель DecisionTreeClassifier**

In [13]:
%%time
model_DTC = DecisionTreeClassifier(max_depth = 4, random_state=42)
model_DTC.fit(features_train, target_train)
predict = model_DTC.predict(features_test)
f1_DTC = f1_score(target_test, predict)
print(f"Показатель f1 на тестовой выборке: {f1_DTC:.2f}")

Показатель f1 на тестовой выборке: 0.94
CPU times: user 1min 1s, sys: 148 ms, total: 1min 1s
Wall time: 1min 1s


**Модель CatBoost**

In [14]:
%%time
model_CBC = CatBoostClassifier(verbose=False, iterations=100)
model_CBC.fit(features_train, target_train)
target_predict = model_CBC.predict(features_test)
f1_CBC = f1_score(target_test, target_predict)

print(f"Показатель f1 на тестовой выборке: {f1_CBC:.2f}")

Показатель f1 на тестовой выборке: 0.94
CPU times: user 2min 5s, sys: 1.21 s, total: 2min 6s
Wall time: 32.7 s


**Вывод** 

В данном пункте были обучены модели LogisticRegression, DecisionTreeClassifier, CatBoost. Результаты F1 данных моделей приведены в таблице ниже:

In [16]:
d = ({ 'Показатель f1 на тестовой выборке':[f1_lr, f1_DTC, f1_CBC]
      })
data = pd.DataFrame(data=d, index=['LogisticRegression','DecisionTreeClassifier','CatBoost'])
data

,Показатель f1 на тестовой выборке
LogisticRegression,0.948016
DecisionTreeClassifier,0.938547
CatBoost,0.943617


По качеству метрики f1 по времени наилучшей моделью оказалась LogisticRegression (f1 = 0.95).

## Выводы

В ходе выполнения проекта о построении модели для выявления токсичных комментариев сделано следующее:

1) подготовлены данные к обучению. С помошью модели BERT произведена токенизация, созданая маска внивания и извлечены эмбеддинги.

2) обучены модели LogisticRegression, DecisionTreeClassifier, CatBoost. По качеству метрики f1 и по времени наилучшей моделью оказалась LogisticRegression (f1 = 0.95).